In [1]:
from components import *
import numpy as np
import scipy.signal as sci
import time

In [2]:
def benchmark(layer, x_shape, delta_shape, samples = 50, resolution = 200, digits_precision = 4):
    def measure_func(func, input_shape):
        times = []

        for _ in range(samples):
            now = time.time()
            for i in range(resolution):
                input = np.random.randn(*input_shape)
                func(input)
            times.append(time.time() - now)

        mean = 1000 * np.mean(times) / resolution
        median = 1000 * np.median(times) / resolution
        std = 1000 * np.std(times) / np.sqrt(resolution)

        return mean, median, std

    mean_ffwd, median_ffwd, std_ffwd = measure_func(layer.feedforward, x_shape)
    mean_back, median_back, std_back = measure_func(layer.backprop, delta_shape)

    print(f"Feedforward: {round(mean_ffwd, digits_precision)} ± {round(std_ffwd, digits_precision)} ms         median {median_ffwd} ms")
    print(f"Backprop: {round(mean_back, digits_precision)} ± {round(std_back, digits_precision)} ms         median {median_back} ms")

In [3]:
get_optimizer = lambda : SGD_momentum_optimizer(0.001, 20, 0.99).get_optimizer()

In [4]:
# FullyConnected benchmark
f = FullyConnected(100, 100, tanh(), L2Regularization(0.001))
f.set_optimizer(get_optimizer())
benchmark(f, (100, 1), (100, 1))

Feedforward: 0.0578 ± 0.4628 ms         median 0.06307363510131836 ms
Backprop: 0.1326 ± 0.6538 ms         median 0.13674676418304443 ms


In [5]:
f = FullyConnected(200, 200, tanh(), L2Regularization(0.001))
f.set_optimizer(get_optimizer())
benchmark(f, (200, 1), (200, 1))

Feedforward: 0.0652 ± 0.4605 ms         median 0.07746338844299316 ms
Backprop: 0.2936 ± 0.7205 ms         median 0.2887493371963501 ms


In [77]:
conv = Convolution((4, 28, 28), (3, 3), 8, tanh())
conv.set_optimizer(get_optimizer())
benchmark(conv, (10, 4, 28, 28), (10, 8, 26, 26), resolution=20)

Feedforward: 35.6962 ± 38.3102 ms         median 33.080703020095825 ms
Backprop: 77.7228 ± 23.1754 ms         median 78.09931635856628 ms


In [4]:
maxpool = MaxPool_Old((4, 28, 28))
benchmark(maxpool, (10, 4, 28, 28), (10, 4, 14, 14), resolution=20)

Feedforward: 115.4514 ± 61.6142 ms         median 117.74694919586182 ms
Backprop: 17.5074 ± 10.4858 ms         median 17.35813617706299 ms


In [8]:
maxpool_new = MaxPool((4, 28, 28))
benchmark(maxpool_new, (10, 4, 28, 28), (10, 4, 14, 14), resolution = 50)

Feedforward: 2.4619 ± 5.4346 ms         median 2.2155141830444336 ms
Backprop: 0.5734 ± 0.8491 ms         median 0.5687475204467773 ms
